# Setup

In [ ]:
# Installs

%pip install -qU langchain-cohere langchain-community langchain-core==0.2.40 langchain_experimental langchain-google-community langchain-google-genai langchain-huggingface==0.0.3 langchain-openai langchain-qdrant langchain-google-vertexai
%pip install -qU -q chainlit==1.1.302
%pip install -qU -q docx2txt
%pip install -qU google-cloud-aiplatform
%pip install -qU google-cloud-discoveryengine
%pip install -qU IProgress
%pip install -qU nltk
%pip install -qU openpyxl
%pip install -qU pymupdf
%pip install -qU python-dotenv
%pip install -qU ragas==0.1.20
%pip install -qU tqdm
%pip install -qU unstructured 

# Verify installed packages have compatible dependencies
%pip check

In [ ]:
# Download punkt_tab module that is used for sentence tokenizaiton

import nltk

nltk.download('punkt_tab')

# Environment Variables

In [ ]:
# Get environment variables

import os
from dotenv import load_dotenv
import uuid

# Load environment variables from .env file
load_dotenv()

# Google Auth

In [ ]:
# Set Google user permissions

import google.auth
import os
import sys

# If running in Colab, use the permissions of the currently authenticated user
if 'google.colab' in sys.modules:
	print('Running in Google Colab')
	from google.colab import auth

	auth.authenticate_user()

# If not, set the GOOGLE_APPLICATION_CREDENTIALS to the service account credentials file 
else:
	print('Running locally')
	# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'credentials.json'
	# print(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

#####

def test_google_perms():
	from google.cloud import storage
	
	creds, _ = google.auth.default(quota_project_id=os.environ["PROJECT_ID"])

	# Now, you can use the Google Cloud client libraries
	client = storage.Client(credentials=creds)

	# List all buckets in your project
	buckets = list(client.list_buckets())
	print(buckets)

test_google_perms()

In [ ]:
# Initialize Vertex AI

import vertexai

vertexai.init(project=os.environ['PROJECT_ID'], location=os.environ['REGION'])

In [ ]:
# Verfify that our Google API key works

import requests

GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
url = f'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={GOOGLE_API_KEY}'
  
headers = {
	"Content-Type": "application/json",
}
  
data = {"contents":[{"parts":[{"text":"What is MassHealth?"}]}]}

response = requests.post(url, headers=headers, json=data)
print(response.text)

# Utils

In [ ]:
# Load the docs from a directory

# https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/file_directory/

from langchain_community.document_loaders import DirectoryLoader 
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredExcelLoader

def process_directory(path: str, glob: str, loader_cls: str, use_multithreading=True):
	
	loader = DirectoryLoader(path=path, glob=glob, show_progress=True, loader_cls=loader_cls, use_multithreading=use_multithreading)
	
	docs = loader.load()
	
	return docs

#####

def test_process_directory():
	docs = []
	
	docs_pdf = process_directory('docs/masshealth/', '**/*.pdf', PyMuPDFLoader, True)
	# print(len(docs_pdf))
	docs_excel = process_directory('docs/masshealth/', '**/*.xlsx', UnstructuredExcelLoader, True)
	# print(len(docs_excel))

	docs.extend(docs_pdf)
	docs.extend(docs_excel)
	print(len(docs))

test_process_directory()

In [ ]:
# Load the docs from a single file

# TODO - add TextLoader refernece
# TODO - add CSVLoader reference
# https://python.langchain.com/docs/integrations/document_loaders/pymupdf/
# https://python.langchain.com/docs/integrations/document_loaders/microsoft_word/

from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import Docx2txtLoader

def process_file(path: str):

	docs = None

	# Select the right loader
	if 'txt' in path.lower():
		loader = TextLoader(path)
	elif 'pdf' in path.lower():
		loader = PyMuPDFLoader(path)
	elif 'docx' in path.lower():
		loader = Docx2txtLoader(path)
	else:
		print(f'No document loader found for {path}')

	docs = loader.load()

	return docs

#####

def test_process_file():
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")
	print(docs[0].metadata)

test_process_file()

In [ ]:
# Create embeddings using Hugging Face 

from langchain_huggingface import HuggingFaceEmbeddings

def create_embeddings_huggingface(model='Snowflake/snowflake-arctic-embed-m') -> HuggingFaceEmbeddings:

	# Initialize the OpenAIEmbeddings class
	embeddings = HuggingFaceEmbeddings(model_name=model)

	return embeddings

#####

def test_create_embeddings_huggingface():
	text = 'What is my benefit for acupuncture?'
	embeddings = create_embeddings_huggingface()
	vector = embeddings.embed_query(text)
	print(vector)
	return embeddings

test_create_embeddings_huggingface()

In [ ]:
# Create embeddings using OpenAI

from langchain_openai import OpenAIEmbeddings

def create_embeddings_openai(model="text-embedding-ada-002") -> OpenAIEmbeddings:

	# Initialize the OpenAIEmbeddings class
	embeddings = OpenAIEmbeddings(model=model)

	return embeddings

#####

def test_create_embeddings_openai():
	text = "What is my benefit for acupuncture?"
	embeddings = create_embeddings_openai()
	vector = embeddings.embed_query(text)
	print(vector)
	return embeddings

test_create_embeddings_openai()

In [ ]:
# Create embeddings using Vertex AI

# https://python.langchain.com/docs/integrations/text_embedding/google_vertex_ai_palm/

from langchain_google_vertexai import VertexAIEmbeddings

def create_embeddings_vertexai(model="text-embedding-004") -> VertexAIEmbeddings:

	creds, _ = google.auth.default(quota_project_id=os.environ["PROJECT_ID"])

	# Initialize the VertexAIEmbeddings class
	embeddings = VertexAIEmbeddings(model_name=model, 
								 credentials=creds)

	return embeddings

#####

def test_create_embeddings_vertexai():
	text = 'What is my benefit for acupuncture?'
	embeddings = create_embeddings_vertexai()
	vector = embeddings.embed_query(text)
	print(vector)
	return embeddings

test_create_embeddings_vertexai()

In [ ]:
# Remove empty chunks 

from langchain.schema import Document

def remove_empty_chunks(chunks_start: list) -> list:
	
	start = len(chunks_start)
	# print(f'start - {start} chunks')
	
	# Remove empty chunks
	chunks_end = [chunk for chunk in chunks_start if chunk.page_content.strip()]

	end = len(chunks_end)
	# print(f'end - {end} chunks')

	return chunks_end   

#####

def test_remove_empty_chunks():
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")

	# Added a test doc
	doc = Document(
		page_content='',
		metadata=docs[0].metadata
	)
	docs.append(doc)
	print(len(docs))

	# Remove the empty doc (chunk)
	docs = remove_empty_chunks(docs)
	print(len(docs))

test_remove_empty_chunks()

In [ ]:
# Create a text splitter using recursive character text splitter

# https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/

from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_docs_recursive(documents: list, chunk_size=500, chunk_overlap=50) -> list:

	text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

	chunks_start = text_splitter.split_documents(documents)

	chunks_end = remove_empty_chunks(chunks_start=chunks_start)

	return chunks_end

#####

def test_chunk_docs_recursive(): 
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")
	chunks = chunk_docs_recursive(documents=docs)
	print(len(chunks))
	print(chunks[0].page_content)

test_chunk_docs_recursive()

In [ ]:
# Create a text splitter using NLTK

# https://python.langchain.com/docs/how_to/split_by_token/

from langchain.text_splitter import NLTKTextSplitter

def chunk_docs_nltk(documents: list, chunk_size=512, chunk_overlap=64) -> list:

	text_splitter = NLTKTextSplitter(
	chunk_size=chunk_size, 
	chunk_overlap=chunk_overlap)

	chunks_start = text_splitter.split_documents(documents)

	chunks_end = remove_empty_chunks(chunks_start=chunks_start)

	return chunks_end

#####

def test_chunk_docs_nltk(): 
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")
	chunks = chunk_docs_nltk(documents=docs)
	print(len(chunks))
	print(chunks[0].page_content)

test_chunk_docs_nltk()


In [ ]:
# Create a text splitter semantic chunking 

# https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/semantic-chunker/

from langchain_experimental.text_splitter import SemanticChunker

def chunk_docs_semantic(documents: list, ) -> list:

	# TODO - Use embeddings parameter
	text_splitter = SemanticChunker(create_embeddings_openai(), breakpoint_threshold_type="percentile")

	chunks_start = text_splitter.split_documents(documents)

	# Remove empty chunks
	chunks_end = remove_empty_chunks(chunks_start)

	return chunks_end

#####

def test_chunk_docs_semantic():
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")
	chunks = chunk_docs_semantic(docs)
	print(len(chunks))
	print(chunks[0].page_content)

test_chunk_docs_semantic()

In [ ]:
# Create a Qdrant vector store

from langchain_core.embeddings import Embeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

def create_qdrant_vector_store(location: str, collection_name: str, vector_size: int, embeddings: Embeddings, documents: list) -> QdrantVectorStore:

	# Initialize the Qdrant client
	qdrant_client = QdrantClient(location=location)

	# Create a collection in Qdrant
	qdrant_client.create_collection(collection_name=collection_name, vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE))

	# Initialize QdrantVectorStore with the Qdrant client
	qdrant_vector_store = QdrantVectorStore(client=qdrant_client, collection_name=collection_name, embedding=embeddings)
	
	qdrant_vector_store.add_documents(documents)
	
	return qdrant_vector_store

#####

def test_create_qdrant_vector_store():
	embeddings = create_embeddings_openai()
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")
	print(len(docs))
	chunks = chunk_docs_recursive(docs)
	print(len(chunks))
	vector_store = create_qdrant_vector_store(":memory:", "masshealth-test", 1536, embeddings, chunks)
	print(vector_store.collection_name)

test_create_qdrant_vector_store()

In [ ]:
# Create a Qdrant retriever

# TODO - Add reference 

from langchain_core.retrievers import BaseRetriever
from langchain_qdrant import QdrantVectorStore

def create_retriever_qdrant(vector_store: QdrantVectorStore) -> BaseRetriever:

	retriever = vector_store.as_retriever()

	return retriever

#####

def test_create_retriever_qdrant(text: str = None):
	embeddings = create_embeddings_openai()
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")
	chunks = chunk_docs_recursive(docs)
	vector_store = create_qdrant_vector_store(":memory:", "masshealth-test", 1536, embeddings, chunks)
	retriever = create_retriever_qdrant(vector_store)
	if text:
		docs = retriever.invoke(text)
		print(docs[0])

print('\nQDRANT')
test_create_retriever_qdrant('What is my benefit for acupuncture?')

In [ ]:
# Create a Vertex AI retriever

# https://python.langchain.com/docs/integrations/retrievers/google_vertex_ai_search/

from langchain_google_community import VertexAISearchRetriever

def create_retriever_vertexai() -> VertexAISearchRetriever:

	retriever = VertexAISearchRetriever(project_id=os.environ['PROJECT_ID'], 
									 location_id=os.environ['LOCATION_ID'], 
									 data_store_id=os.environ['DATA_STORE_ID'], 
									 max_documents=3)

	return retriever

#####

def test_create_retriever_vertexai(text: str = None):
	retriever = create_retriever_vertexai()
	if text:
		docs = retriever.invoke(text)
		print(docs[0])

print('\nVERTEX AI')
test_create_retriever_vertexai('What is my benefit for acupuncture?')

In [ ]:
# Create a prompt template

# https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/#chatprompttemplate
# https://python.langchain.com/v0.2/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

from langchain.prompts import ChatPromptTemplate

def create_chat_prompt_template(template: str = None) -> ChatPromptTemplate:
	
	if template is None:
		template = '''
		You are a helpful conversational agent for the State of California.
		Your expertise is fully understanding the Medi-Cal provider manuals. 
		You need to answer questions posed by a member, who is trying to get answers about services provided by Medi-Cal.  
		Your goal is to provide a helpful and detailed response, in at least 2-3 sentences. 

		You will be analyzing the health plan documents to derive a good answer, based on the following information:
		1. The question asked.
		2. The provided context, which comes from health plan document. You will need to answer the question based on the provided context and the conversation history.

		Now it's your turn!

		{question}

		{context}
		'''
	
	prompt = ChatPromptTemplate.from_template(template)

	return prompt

#####

def test_create_chat_prompt_template():
	prompt = create_chat_prompt_template()
	print(prompt)

test_create_chat_prompt_template()

In [ ]:
# Create a Langchain chain..

# https://python.langchain.com/docs/integrations/llms/google_ai/
# https://python.langchain.com/docs/integrations/chat/google_generative_ai/
# https://ai.google.dev/gemini-api/docs/safety-settings 

from google.generativeai.types import HarmCategory, HarmBlockThreshold
from langchain_core.output_parsers import StrOutputParser
from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableSerializable
from langchain_google_genai import ChatGoogleGenerativeAI
from operator import itemgetter

def create_chain (model_name: str, 
				  prompt_template: ChatPromptTemplate, 
				  retriever: BaseRetriever) -> RunnableSerializable:

	if "gemini" in model_name.lower():
		llm = ChatGoogleGenerativeAI(
			model=model_name,
			temperature=0,
			safety_settings={
					HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
					HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
					HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
					HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
				},
			)
	else:
		print("Unsuported model name")
		
	chain = (
		{"context": itemgetter("question") | retriever, "question": itemgetter("question")} 
		| RunnablePassthrough.assign(context=itemgetter("context")) 
		| {"response": prompt_template | llm, "context": itemgetter("context")}
		)

	return chain

#####

def test_create_chain_qdrant():
	embeddings = create_embeddings_openai()
	docs = process_file('docs/medi-cal/acu.pdf')
	chunks = chunk_docs_recursive(docs)
	vector_store = create_qdrant_vector_store(":memory:", "test", 1536, embeddings, chunks)
	retriever = create_retriever_qdrant(vector_store)
	chat_prompt_template = create_chat_prompt_template()
	chain = create_chain('gemini-1.5-flash', chat_prompt_template, retriever)
	result = chain.invoke({'question' : 'What is my benefit for acupuncture?'})
	print(result)

print('\nQDRANT')
test_create_chain_qdrant()

###

def test_create_chain_vertexai():
	retreiver = create_retriever_vertexai()
	chat_prompt_template = create_chat_prompt_template()
	chain = create_chain('gemini-1.5-flash', chat_prompt_template, retreiver)
	result = chain.invoke({'question' : 'What is my benefit for acupuncture?'})
	print(result)

print('\nVERTEX AI')
test_create_chain_vertexai()

In [ ]:
# Generate answers from a chain usin a list of questions

import json
from langchain_core.runnables import RunnableSerializable
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser


def generate_answers_contexts(chain: RunnableSerializable, 
							  questions: list):
	
	answers = []
	contexts = []

	# Loop over the list of questions and call the chain to get the answer and context
	for question in questions:
		print(question)

		# Call the chain to get answers and contexts
		response = chain.invoke({"question" : question})
		print(response)

		# Capture the answer and context 
		answers.append(response["response"].content)
		contexts.append([context.page_content for context in response["context"]])

	return answers, contexts

#####

def test_generate_answers_contexts():
	retreiver = create_retriever_vertexai()
	chat_prompt_template = create_chat_prompt_template()
	chain = create_chain(model_name="gemini-1.5-flash", 
					  prompt_template=chat_prompt_template,
					  retriever=retreiver)
	
	questions = ["What is my benefit for acupuncture?",
			  "Who should I call if I have an emergency?",
			  "Provide an overview of the newborn hearing screening program?"]
	
	answers, contexts = generate_answers_contexts(chain=chain, questions=questions)
	
	print(f"Total number of answers = {len(answers)}")
	print(f"Total number of contexts = {len(contexts)}")

test_generate_answers_contexts()

In [ ]:
# Run a Ragas evaluation 

from datasets import Dataset
from langchain_core.runnables import RunnableSerializable
from pandas import DataFrame
from ragas import evaluate
from ragas.metrics import (faithfulness, answer_relevancy, answer_correctness, context_recall, context_precision)

def run_ragas_evaluation(chain: RunnableSerializable, 
						 questions: list, 
						 groundtruths: list, 
						 eval_metrics: list = [answer_correctness, answer_relevancy, context_recall, context_precision, faithfulness]):
	
	answers = []
	contexts = []
	answers, contexts = generate_answers_contexts(chain=chain, 
											   questions=questions)

	# Create the input dataset 
	input_dataset = Dataset.from_dict({
	"question" : questions,         # From the dataframe
	"answer" : answers,             # From the chain
	"contexts" : contexts,          # From the chain
	"ground_truth" : groundtruths   # From the dataframe
	})

	# Run the Ragas evaluation using the input dataset and eval metrics
	ragas_results = evaluate(input_dataset, eval_metrics)
	ragas_results_df = ragas_results.to_pandas()
	
	return ragas_results, ragas_results_df
	
#####

def test_run_ragas_evaluation():
	print("test")    

test_run_ragas_evaluation()

In [ ]:
# Query Vertex AI Search datastore using HTTP Post

import json
import google.auth
from google.auth.transport.requests import Request
import requests

def query_chunks(query: str, 
				 n: int=5):
		
	PROJECT_ID = os.environ['PROJECT_ID']
	LOCATION_ID = os.environ['LOCATION_ID']
	DATA_STORE_ID = os.environ['DATA_STORE_ID']

	if LOCATION_ID == 'us':
		api_endpoint = 'us-discoveryengine.googleapis.com'
	else:
		api_endpoint = 'discoveryengine.googleapis.com'

	url = f"https://{api_endpoint}/v1alpha/projects/{PROJECT_ID}/locations/{LOCATION_ID}/collections/default_collection/dataStores/{DATA_STORE_ID}/servingConfigs/default_search:search"
	print(url)
	
	headers = {
			"Authorization": f"Bearer {access_token}",
			"Content-Type": "application/json",
	}
	
	post_data = {
			"servingConfig": f"projects/{PROJECT_ID}/locations/{LOCATION_ID}/collections/default_collection/dataStores/{DATA_STORE_ID}/servingConfigs/default_search",
			"pageSize": n,
			"query": query,
			"contentSearchSpec": {"searchResultMode": "CHUNKS"},
	}
	
	response = requests.post(url, headers=headers, json=post_data)

	if response.status_code != 200:
		print(
				f"Error retrieving search results: {response.status_code} -"
				f" {response.text}"
		)

	return response.json()

#####

def test_query_chunks():
	creds, _ = google.auth.default()
	creds.refresh(Request())
	access_token = creds.token
	# print(access_token)
	response = query_chunks('What is my benefit for acupuncture?')
	print(json.dumps(response, indent=4))

test_query_chunks()

# Create RAG chain using Google Vertex AI

In [ ]:
# Build RAG chain using Vertex AI Agent Builder datastore

retreiver = create_retriever_vertexai()
chat_prompt_template = create_chat_prompt_template()
chain = create_chain(model_name='gemini-1.5-flash', 
                     prompt_template=chat_prompt_template, 
                     retriever=retreiver)

In [ ]:
# Test the chain 

questions = ['What is my benefit for acupuncture?',
'Who should I call if I have an emergency?',
'Provide an overview of the newborn hearing screening program?',]

for question in questions:
	print(question)
	result = chain.invoke({"question" : question})
	print(result)
	print(result["response"].content)

# Create synthetic testset

In [ ]:
# https://docs.ragas.io/en/latest/references/generate/

import google.auth
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas.testset.generator import RunConfig
from ragas.testset.generator import TestsetGenerator

testset_name = "masshealth_testset.csv"
use_chunking = False
use_vertexai = True

# Load the docs
docs = []
docs_pdf = process_directory('docs/masshealth/', '**/*.pdf', PyMuPDFLoader, True)
docs.extend(docs_pdf)
print(len(docs))

if use_chunking:
	# Chunk the docs 
	docs = chunk_docs_recursive(docs)
	# docs = chunk_docs_nltk(docs)
	# docs = chunk_docs_semantic(docs)

# Set up the parameters for generating the testset
if use_vertexai:
	testset_name = "masshealth_vertexai_testset_2.csv"

	# https://docs.ragas.io/en/v0.1.21/howtos/customisations/gcp-vertexai.html
	creds, _ = google.auth.default(quota_project_id=os.environ["PROJECT_ID"])
	generator_llm = ChatVertexAI(credentials=creds,
							  model_name="gemini-1.5-pro")
	critic_llm = ChatVertexAI(credentials=creds,
							model_name="gemini-1.0-pro")
	embeddings = VertexAIEmbeddings(credentials=creds, 
								 model_name="textembedding-gecko")
else:
	testset_name = "masshealth_openai_testset.csv"

	generator_llm = ChatOpenAI(model="gpt-4o")
	critic_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
	embeddings = OpenAIEmbeddings()

# https://docs.ragas.io/en/v0.1.21/howtos/customisations/run_config.html
run_config=RunConfig(
	max_workers=2	# default: 16 but smaller number is required to avoid rate limits
	)

generator = TestsetGenerator.from_langchain(generator_llm=generator_llm, 
											critic_llm=critic_llm, 
											embeddings=embeddings)

test_size=10

distributions = {simple: 0.5, 
				 multi_context: 0.4, 
				 reasoning: 0.1}

# Generate the testset and save to disk 
testset = generator.generate_with_langchain_docs(documents=docs, 
												 test_size=test_size, 
												 distributions=distributions,
												 run_config=run_config)

# Write the testet to disk 
print(f"Writing {testset_name}")
testset_df = testset.to_pandas()
testset_df.to_csv(f"testsets/{testset_name}")
testset_df

# Evaluate synthetic dataset using Ragas

In [ ]:
# Evaluate the synthetic testset using Ragas

import pandas as pd

# Run the Ragas evaluation and show the results
# Get the questions and groundtruths from the dataframe
testset_df = pd.read_csv("testsets/synthetic_testset.csv")

questions = testset_df["question"].values.tolist()
questions = [str(question) for question in questions]

groundtruths = testset_df["ground_truth"].values.tolist()
groundtruths = [str(ground_truth) for ground_truth in groundtruths]  

# Specify the eval metrics
eval_metrics = [answer_correctness, answer_relevancy, context_precision, context_recall, faithfulness]

# Run the Ragas evaluation and show the results
ragas_results, ragas_results_df = run_ragas_evaluation(chain, questions, groundtruths, eval_metrics)

# Write the results to disk
print("Writing synthetic_testset_ragas_results.csv")
ragas_results_df.to_csv("ragas/synthetic_testset_ragas_results.csv")

# Show the resutls
ragas_results

# Create sample ReAct agent using Gemini

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_google_genai import ChatGoogleGenerativeAI
from google.generativeai.types import HarmCategory, HarmBlockThreshold

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", 
							 temperature=0, 
							 safety_settings={
								 HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
								 HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
								 HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
								 HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH
								 }
								 )

tools = load_tools(["arxiv"])

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What's the paper 1605.08386 about?"})

# Use Unstructured to parse docs and load to Faiss

In [ ]:
# https://colab.research.google.com/drive/1BJYYyrPVe0_9EGyXqeNyzmVZDrCRZwsg?usp=sharing#scrollTo=CwzrH-9_K6-z

%pip install -qU "unstructured-ingest[pdf]" unstructured faiss-cpu

In [ ]:
from unstructured_ingest.v2.pipeline.pipeline import Pipeline
from unstructured_ingest.v2.interfaces import ProcessorConfig
from unstructured_ingest.v2.processes.connectors.local import (
	LocalIndexerConfig,
	LocalDownloaderConfig,
	LocalConnectionConfig,
	LocalUploaderConfig
)
from unstructured_ingest.v2.processes.partitioner import PartitionerConfig
from unstructured_ingest.v2.processes.chunker import ChunkerConfig

In [ ]:
import os 

directory_with_pdfs="app_2/content/data"
directory_with_results="app_2/content/output"

In [ ]:
Pipeline.from_configs(
	context=ProcessorConfig(),
	indexer_config=LocalIndexerConfig(input_path=directory_with_pdfs),
	downloader_config=LocalDownloaderConfig(),
	source_connection_config=LocalConnectionConfig(),
	partitioner_config=PartitionerConfig(
		partition_by_api=True,
		api_key=os.getenv("UNSTRUCTURED_API_KEY"),
		partition_endpoint=os.getenv("UNSTRUCTURED_API_URL"),
		strategy="hi_res",
		additional_partition_args={
			"split_pdf_page": True,
			"split_pdf_concurrency_level": 15,
			},
		),
	uploader_config=LocalUploaderConfig(output_dir=directory_with_results)
).run()

In [ ]:
from unstructured.staging.base import elements_from_json

elements = []
for filename in os.listdir(directory_with_results):
	if filename.endswith('.json'):
		file_path = os.path.join(directory_with_results, filename)
		try:
			elements.extend(elements_from_json(filename=file_path))
		except IOError:
			print(f"Error: Could not read file {filename}.")
print(len(elements))

In [ ]:
from langchain.schema import Document

docs = []

for element in elements:
	metadata = element.metadata.to_dict()
	docs.append(Document(page_content=element.text, metadata=metadata))

print(len(docs))

In [ ]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(documents=docs, embedding=create_embeddings_vertexai())
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
vector_store.save_local("app_2/faiss_index_unstructured")

In [ ]:
new_vector_store = FAISS.load_local("app_2/faiss_index_unstructured", create_embeddings_vertexai(), allow_dangerous_deserialization=True)
new_retriever = new_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
new_chain = create_chain('gemini-1.5-flash', chat_prompt_template, new_retriever)

In [ ]:
questions = ['What is my benefit for acupuncture?',
'Who should I call if I have an emergency?',
'What is my vision benefit?',]

for question in questions:
	print(question)
	result = chain.invoke({"question" : question})
	print(result)
	print(result["response"].content)

# Using PyMuPDFLoader to parse docs and load to Faiss

In [ ]:
docs = []

docs_pdf = process_directory('docs/medi-cal/', '**/*.pdf', PyMuPDFLoader, True)
docs_excel = process_directory('docs/medi-cal/', '**/*.xlsx', UnstructuredExcelLoader, True)

docs.extend(docs_pdf)
docs.extend(docs_excel)

print(len(docs))



In [ ]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(documents=docs, embedding=create_embeddings_vertexai())
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
vector_store.save_local("app_2/faiss_index_pymupdfloader")

In [ ]:
new_vector_store = FAISS.load_local("app_2/faiss_index_pymupdfloader", create_embeddings_vertexai(), allow_dangerous_deserialization=True)
new_retriever = new_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=new_retriever)

In [ ]:
template = '''
You are a helpful conversational agent for the State of California.
Your expertise is fully understanding the Medi-Cal provider manuals. 
You need to answer questions posed by a member, who is trying to get answers about services provided by Medi-Cal.  
Your goal is to provide a helpful and detailed response, in at least 2-3 sentences. 

You will be analyzing the health plan documents to derive a good answer, based on the following information:
1. The question asked.
2. The provided context, which comes from health plan document. You will need to answer the question based on the provided context and the conversation history.

Now it's your turn!

{question}

{context}
'''

prompt = PromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(
	model='gemini-1.5-flash',
	temperature=0,
	safety_settings={
			HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
			HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
			HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
			HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
		}
	)
		
# new_chain = (
#         {"context": itemgetter("question") | new_retriever, "question": itemgetter("question")} 
#         | RunnablePassthrough.assign(context=itemgetter("context")) 
#         | {"response": prompt | llm, "context": itemgetter("context")}
#         )

new_chain = (
		{"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")} 
		| RunnablePassthrough.assign(context=itemgetter("context")) 
		| {"response": prompt | llm, "context": itemgetter("context")}
		)


In [ ]:
questions = ['What is my benefit for acupuncture?',
'Who should I call if I have an emergency?',
'What is my vision benefit?',]

for question in questions:
	print(question)
	result = new_chain.invoke({"question" : question})
	print(result)
	print(result["response"].content)